In [ ]:
%load_ext autoreload
%autoreload 2

# Proyecto final Bioinformática (NLP)

Mirar el siguiente video. Sobre todo la primer parte como para tener contexto sobre descubrimiento de drogas:

https://www.youtube.com/watch?v=jBlTQjcKuaY

Resumen y puntos importantes del video:
- Queremos entender la bioactividad de una molecúla (molecule_chembl_id) sobre una encima (Acetylcholinesterase)
- La bioactividad se medirá en este caso con el IC50 (standard_value)
- A menor IC50, menos droga para generar la misma actividad, es decir, mayor actividad relativa
- La notación de la fórmula química se llama smiles (https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)
- Existen distintas técnicas para obtener features de las moléculas y en el video se describen 2:
    - Descriptores de Lipinski
    - Fingerprints del tipo pubchem
- Se construye un modelo de regresión con RandomForest para estimar el pI50 (IC50 en escala logarítimica) dado los fingerprints de entrada

### Objetivos del proyecto:
- Evaluar distintas alternativas de modelos de deep learning para resolver este problema
    - LSTM
    - CNN
    - TextCNN
- Mejorar la métrica del RandomForest
- En vez de ingresar con los features de entrada (fingerprints) como en el video, utilizar técnicas de embeddings usuales en NLP
    - Tokenización en modo caracter dado el smiles de la fórmula química
    - Utilizando un tokenizer sobre los smiles
    - Puedo usar técnicas modernas de tokenización (https://deepchem.readthedocs.io/en/2.4.0/api_reference/tokenizers.html)
- La salida a estimar por el modelo será el pIC50
- La métrica, para comparar con los resultados del RandomForest será el $R^2$

### Librerías:
- chembl-webresource-client: Para bajar el dataset (https://pypi.org/project/chembl-webresource-client/)
- deepchem: libería muy interesante con muchas implementaciones de deep learning aplicadas a la química (https://github.com/deepchem/deepchem)

In [ ]:
!pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 596 kB 9.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.1.2 which is incompatible.


In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

# Cliente API

En esta notebook solo se baja el dataset. No tiene que hacer nada más que ejecutarla y entenderla

Librería para baja el dataset

In [ ]:
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)

In [ ]:
targets.head()

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P22303', 'xref_name': None, 'xre...",Homo sapiens,Acetylcholinesterase,27.0,False,CHEMBL220,"[{'accession': 'P22303', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Homo sapiens,Cholinesterases; ACHE & BCHE,27.0,False,CHEMBL2095233,"[{'accession': 'P06276', 'component_descriptio...",SELECTIVITY GROUP,9606
2,[],Drosophila melanogaster,Acetylcholinesterase,17.0,False,CHEMBL2242744,"[{'accession': 'P07140', 'component_descriptio...",SINGLE PROTEIN,7227
3,"[{'xref_id': 'P04058', 'xref_name': None, 'xre...",Torpedo californica,Acetylcholinesterase,15.0,False,CHEMBL4780,"[{'accession': 'P04058', 'component_descriptio...",SINGLE PROTEIN,7787
4,"[{'xref_id': 'P21836', 'xref_name': None, 'xre...",Mus musculus,Acetylcholinesterase,15.0,False,CHEMBL3198,"[{'accession': 'P21836', 'component_descriptio...",SINGLE PROTEIN,10090


In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target


'CHEMBL220'

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

# Bajada de data
Puede tardar un poco dependiendo de que tan saturado este el server

Por eso el for, para ver el progreso y bajar la ansiedad. Son en el orden de 7500K

In [ ]:
res_cols = []
for i, r in enumerate(res):
    print(f'{i}\r', end='')
    res_cols.append(r)

In [ ]:
df = pd.DataFrame(res_cols)

In [ ]:
df.shape

(8205, 45)

In [ ]:
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,33969,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.75
1,None,37563,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.1
2,None,37565,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,50.0
3,None,38902,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.3
4,None,41170,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8200,None,22970946,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4770653,Inhibition of AChE (unknown origin) preincubat...,B,None,None,BAO_0000179,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,None,None,0.25
8201,None,22970947,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4770653,Inhibition of AChE (unknown origin) preincubat...,B,None,None,BAO_0000179,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,None,None,0.03
8202,None,22986924,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4774393,Inhibition of human erythrocytes AChE using ac...,B,None,None,BAO_0000179,BAO_0000019,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,None,None,28.98
8203,None,22986925,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4774393,Inhibition of human erythrocytes AChE using ac...,B,None,None,BAO_0000179,BAO_0000019,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,None,None,0.0064


# Limpio data

In [ ]:
df = df.dropna(subset=['standard_value', 'canonical_smiles'])
df = df.drop_duplicates(['canonical_smiles'])

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df = df[selection]

In [ ]:
df

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0
...,...,...,...
8195,CHEMBL4779440,Brc1ccc([N+]2=Cc3ccccc3CC2)cc1.[Br-],10500.0
8196,CHEMBL417799,C[n+]1cc2c3c(ccc2c2ccc4cc5c(cc4c21)OCO5)OCO3,1220.0
8197,CHEMBL13045,COc1ccc2c(c[n+](C)c3c4cc5c(cc4ccc23)OCO5)c1OC,1450.0
8202,CHEMBL3085398,C=CC(C)(C)c1cc([C@@H]2CC(=O)c3c(O)cc(O)c(CC=C(...,28980.0


# Preprocesamiento y normalización

In [ ]:
import numpy as np

In [ ]:
df['standard_value'] = df['standard_value'].apply(pd.to_numeric)

In [ ]:
# En el video se hace esta normalización. No la veo del todo necesaria
df['standard_value_norm'] = df['standard_value'].apply(lambda x: (x>1e8)*1e8 + (x<=1e8)*x)

In [ ]:
df['pIC50'] = df['standard_value'].apply(lambda x: -np.log10(x*(10**-9))) 

In [ ]:
df

,molecule_chembl_id,canonical_smiles,standard_value,standard_value_norm,pIC50
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0,750.0,6.124939
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0,100.0,7.000000
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0,50000.0,4.301030
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0,300.0,6.522879
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0,800.0,6.096910
...,...,...,...,...,...
8195,CHEMBL4779440,Brc1ccc([N+]2=Cc3ccccc3CC2)cc1.[Br-],10500.0,10500.0,4.978811
8196,CHEMBL417799,C[n+]1cc2c3c(ccc2c2ccc4cc5c(cc4c21)OCO5)OCO3,1220.0,1220.0,5.913640
8197,CHEMBL13045,COc1ccc2c(c[n+](C)c3c4cc5c(cc4ccc23)OCO5)c1OC,1450.0,1450.0,5.838632
8202,CHEMBL3085398,C=CC(C)(C)c1cc([C@@H]2CC(=O)c3c(O)cc(O)c(CC=C(...,28980.0,28980.0,4.537902


In [ ]:
! mkdir data

In [ ]:
print(df['pIC50'].describe())
max_pIC50 =df['pIC50'].max()
id_max_pIC50=df['pIC50'].argmax()

count    5664.000000
mean             inf
std              NaN
min        -6.770000
25%         4.769551
50%         5.626170
75%         6.843527
max              inf
Name: pIC50, dtype: float64


En el describe vemos que figura un valor infinito una vez que se aplica la transformaciòn logaritmica al target por lo cual lo que podemos hacer es agregarle un epsilon ( + K.epsilon() ) algun valor de ( 1*(10**-9) ) muy cercano a 0 o elimino el registro 7994 q es el que me da infinito

In [ ]:
df.iloc[[id_max_pIC50]]

,molecule_chembl_id,canonical_smiles,standard_value,standard_value_norm,pIC50
7994,CHEMBL4780352,COc1cc2c(cc1OC)C(=O)/C(=C/c1ccc(OCCCCN[N+]3(C)...,0.0,0.0,inf


In [ ]:
df.drop(7994,axis=0, inplace=True)
df.describe()

,standard_value,standard_value_norm,pIC50
count,5.663000e+03,5.663000e+03,5663.000000
mean,2.662073e+12,2.859802e+05,5.809503
std,1.083197e+14,4.301615e+06,1.646669
min,5.000000e-06,5.000000e-06,-6.770000
25%,1.435050e+02,1.435050e+02,4.769551
50%,2.370000e+03,2.370000e+03,5.625252
75%,1.700000e+04,1.700000e+04,6.843133
max,5.888437e+15,1.000000e+08,14.301030


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PATH=r"/content/gdrive/MyDrive/bioinformatics_final_project-main/"

In [ ]:
df.to_csv(PATH +'data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)